# Modelo do transporte - parte 1

## Importação da biblioteca

In [1]:
import pyomo.environ as pyo

## Definição dos dados de entrada

In [2]:
custos = [[12, 22, 30], 
          [18, 24, 32], 
          [22, 15, 34]]

capacidade = [100, 140, 160]

demanda = [120, 130, 150]

m = len(capacidade)
n = len(demanda)

## Modelo computacional

In [3]:
modelo = pyo.ConcreteModel()

### Conjuntos

$I \colon \text{Conjunto de distribuidores,} \; I = \{1,2,\ldots,m\},$

$J \colon \text{Conjunto de consumidores,} \; J = \{1,2,\ldots,n\}.$

In [4]:
modelo.I = pyo.RangeSet(m)

modelo.J = pyo.RangeSet(n)

### Parâmetros

$c_{ij} \colon \text{Custo unitário de transporte do distribuidor }i \in I \text{ para o consumidor }j \in J,$

$a_i \colon \text{Capacidade de fornecimento do distribuidor }i \in I$,

$b_j \colon \text{Demanda do consumidor }j \in J.$

In [5]:
modelo.c = pyo.Param(modelo.I, modelo.J, initialize=lambda modelo, i, j: custos[i-1][j-1])

modelo.a = pyo.Param(modelo.I, initialize=lambda modelo, i: capacidade[i-1])

modelo.b = pyo.Param(modelo.J, initialize=lambda modelo, j: demanda[j-1])

### Variáveis de decisão

$x_{ij} \colon \text{Quantidade a ser transportada do distribuidor }i \in I \text{ para o consumidor }j \in J.$

In [6]:
modelo.x = pyo.Var(modelo.I, modelo.J, within=pyo.NonNegativeReals)

### Função objetivo

$\text{min }z(x) = \sum_\limits{i \in I} \sum_\limits{j \in J} c_{ij} x_{ij}.$

In [7]:
def regra_z(mod):
    return pyo.summation(mod.c, mod.x)

modelo.z = pyo.Objective(rule=regra_z, sense=pyo.minimize) # minimize = default

### Restrições

#### Capacidade

$\sum_\limits{j \in J} x_{ij} = a_i, \;\; \forall i \in I,$

In [8]:
def regra_capacidade(mod, i):
    return sum(mod.x[i,j] for j in mod.J) == mod.a[i]

modelo.restr_capacidade = pyo.Constraint(modelo.I, rule=regra_capacidade)

#### Demanda

$\sum_\limits{i \in I} x_{ij} = b_j, \;\; \forall j \in J,$

In [9]:
def regra_demanda(mod, j):
    return sum(mod.x[i,j] for i in mod.I) == mod.b[j]

modelo.restr_demanda = pyo.Constraint(modelo.J, rule=regra_demanda)

#### Não negatividade

$x_{ij} \geq 0 \;\; \forall i \in I, j \in J.$

## Impressão do modelo

In [10]:
modelo.pprint()

2 Set Declarations
    c_index : Size=1, Index=None, Ordered=True
        Key  : Dimen : Domain : Size : Members
        None :     2 :    I*J :    9 : {(1, 1), (1, 2), (1, 3), (2, 1), (2, 2), (2, 3), (3, 1), (3, 2), (3, 3)}
    x_index : Size=1, Index=None, Ordered=True
        Key  : Dimen : Domain : Size : Members
        None :     2 :    I*J :    9 : {(1, 1), (1, 2), (1, 3), (2, 1), (2, 2), (2, 3), (3, 1), (3, 2), (3, 3)}

2 RangeSet Declarations
    I : Dimen=1, Size=3, Bounds=(1, 3)
        Key  : Finite : Members
        None :   True :   [1:3]
    J : Dimen=1, Size=3, Bounds=(1, 3)
        Key  : Finite : Members
        None :   True :   [1:3]

3 Param Declarations
    a : Size=3, Index=I, Domain=Any, Default=None, Mutable=False
        Key : Value
          1 :   100
          2 :   140
          3 :   160
    b : Size=3, Index=J, Domain=Any, Default=None, Mutable=False
        Key : Value
          1 :   120
          2 :   130
          3 :   150
    c : Size=9, Index=c_in

## Resolução

In [11]:
resultado = pyo.SolverFactory('glpk').solve(modelo)

In [12]:
resultado.write()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 8370.0
  Upper bound: 8370.0
  Number of objectives: 1
  Number of constraints: 7
  Number of variables: 10
  Number of nonzeros: 19
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.015750885009765625
# ----------------------------------------------------------
#   Solution Information
# --------------------------------

In [15]:
modelo.x.pprint()

x : Size=9, Index=x_index
    Key    : Lower : Value : Upper : Fixed : Stale : Domain
    (1, 1) :     0 : 100.0 :  None : False : False : NonNegativeReals
    (1, 2) :     0 :   0.0 :  None : False : False : NonNegativeReals
    (1, 3) :     0 :   0.0 :  None : False : False : NonNegativeReals
    (2, 1) :     0 :  20.0 :  None : False : False : NonNegativeReals
    (2, 2) :     0 :   0.0 :  None : False : False : NonNegativeReals
    (2, 3) :     0 : 120.0 :  None : False : False : NonNegativeReals
    (3, 1) :     0 :   0.0 :  None : False : False : NonNegativeReals
    (3, 2) :     0 : 130.0 :  None : False : False : NonNegativeReals
    (3, 3) :     0 :  30.0 :  None : False : False : NonNegativeReals


In [16]:
modelo.z()

8370.0